In [1]:
# Cargamos las librerias que necesitamos
import json
import pandas as pd 
#! pip install nltk
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
ruta_archivo = 'australian_users_items.json'

# Listas para almacenar los datos y los errores
datos_items = []
errores = []

# Contadores para los registros correctos y los registros fallidos
registros_correctos = 0
registros_fallidos = 0

with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
    # Leer cada linea del archivo
    for num_linea, linea in enumerate(archivo, start=1):
        try:
            # Intentar cargar cada línea como un JSON
            objeto_json = json.loads(linea)
            datos_items.append(objeto_json)
            registros_correctos += 1
        except json.JSONDecodeError:
            try:
                # Si falla como JSON, intentar interpretar la línea como un diccionario
                objeto_dict = eval(linea)
                if isinstance(objeto_dict, dict):
                    datos_items.append(objeto_dict)
                    registros_correctos += 1
                else:
                    errores.append(f"Línea {num_linea}: No es un JSON válido ni un diccionario.")
                    registros_fallidos += 1
            except Exception as e:
                errores.append(f"Línea {num_linea}: {e}")
                registros_fallidos += 1

# Creamos un df con datos_items
df_items = pd.DataFrame(datos_items)

# Ahora vemos cuantos correctos y errores se cargaron
print(f"Total de registros: {registros_correctos + registros_fallidos}")
print(f"Registros leídos correctamente: {registros_correctos}")
print(f"Registros fallidos: {registros_fallidos}")
print("Errores:")
for error in errores:
    print(error)

Total de registros: 88310
Registros leídos correctamente: 88310
Registros fallidos: 0
Errores:


In [3]:
df_items.info()
#vemos un df completo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [4]:
# Creamos una lista para guardar los df resultantes.
dataframes_resultantes = []

# Iterar en cada fila del df.
for index, row in df_items.iterrows():
    lista_diccionarios = row['items']
    
    # Convierte la lista de diccionarios en un df.
    df_temporal = pd.DataFrame(lista_diccionarios)
    
    # Agrega el df resultante a la lista.
    dataframes_resultantes.append(df_temporal)

# dataframes_resultantes tendra df separados, uno por cada fila de la columna 'columna_lista_diccionarios'.


In [5]:
# aca unifico los df en un solo 
df_unificado = pd.concat(dataframes_resultantes, ignore_index=True)

In [6]:
df_unificado.head(25)

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0
5,60,Ricochet,0,0
6,70,Half-Life,0,0
7,130,Half-Life: Blue Shift,0,0
8,300,Day of Defeat: Source,4733,0
9,240,Counter-Strike: Source,1853,0


In [7]:
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
dtypes: int64(2), object(2)
memory usage: 157.3+ MB


In [8]:
df_items_desanidado = pd.concat([df_items, df_unificado], axis=1) 



In [9]:
df_items_desanidado.head()

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",10,Counter-Strike,6,0
1,js41637,888.0,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...",20,Team Fortress Classic,0,0
2,evcentric,137.0,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...",30,Day of Defeat,7,0
3,Riot-Punch,328.0,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...",40,Deathmatch Classic,0,0
4,doctr,541.0,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...",50,Half-Life: Opposing Force,0,0


In [10]:
df_items_desanidado = df_items_desanidado.drop('items', axis=1)

In [13]:
df_items_desanidado.head()

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever
0,76561197970982479,277.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",10,Counter-Strike,6
1,js41637,888.0,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...",20,Team Fortress Classic,0
2,evcentric,137.0,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...",30,Day of Defeat,7
3,Riot-Punch,328.0,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...",40,Deathmatch Classic,0
4,doctr,541.0,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...",50,Half-Life: Opposing Force,0


In [11]:
df_items_desanidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       float64
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  int64  
 7   playtime_2weeks   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 314.5+ MB


In [12]:
# Esto proporcionará la cantidad de valores no nulos por columna.
count_per_column = df_items_desanidado.count()
print(count_per_column)


user_id               88310
items_count           88310
steam_id              88310
user_url              88310
item_id             5153209
item_name           5153209
playtime_forever    5153209
playtime_2weeks     5153209
dtype: int64


In [13]:
df_items_desanidado.isnull().sum()

user_id             5064899
items_count         5064899
steam_id            5064899
user_url            5064899
item_id                   0
item_name                 0
playtime_forever          0
playtime_2weeks           0
dtype: int64

In [14]:
df_items_desanidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       float64
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  int64  
 7   playtime_2weeks   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 314.5+ MB


In [15]:
df_items_desanidado.dropna(inplace=True)


In [16]:
df_items_desanidado.isnull().sum()

user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

In [17]:
#Exportamos df
nombre_del_archivo = 'itemscompleto.csv'
df_items_desanidado.to_csv(nombre_del_archivo, index=False)  # El argumento index=False evita que se escriba el índice en el archivo.

print(f"Se ha exportado el DataFrame a {nombre_del_archivo}")


Se ha exportado el DataFrame a itemscompleto.csv
